<a href="https://colab.research.google.com/github/manuelpope/DsPalermoPractica/blob/master/SDGModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
from sklearn.linear_model import SGDClassifier
import pandas as pd
import numpy as np






In [118]:
path= '/content/drive/My Drive/Colab Notebooks/cleanDatavfromColab.csv'
data = pd.read_csv(path)
data.head()
data['name'].count()

294121

In [119]:
#split the data frame test y train 
y= data['success']
listLenghts= []
for i in range(0,data['name'].count()):
  listLenghts.append(len(data['name'].iloc[i]))
data['name']= listLenghts

X = data.drop(columns=['Unnamed: 0','success', 'ID','usd pledged','state','pledged','launched','deadline','inUSDPledged'])
X.head(10)



,name,category,main_category,currency,goal,backers,country,inUSDGoal,gap,range
0,31,Poetry,Publishing,GBP,1000.0,0,GB,1240.0,58,muy pequeño
1,14,Narrative Film,Film & Video,USD,45000.0,3,US,45000.0,45,muy grande
2,49,Music,Music,USD,5000.0,1,US,5000.0,30,mediano
3,58,Film & Video,Film & Video,USD,19500.0,14,US,19500.0,55,grande
4,20,Restaurants,Food,USD,50000.0,224,US,50000.0,35,muy grande
5,60,Food,Food,USD,1000.0,16,US,1000.0,20,muy pequeño
6,49,Drinks,Food,USD,25000.0,40,US,25000.0,44,muy grande
7,53,Product Design,Design,USD,125000.0,58,US,125000.0,35,muy grande
8,57,Documentary,Film & Video,USD,65000.0,43,US,65000.0,30,muy grande
9,19,Nonfiction,Publishing,CAD,2500.0,0,CA,1850.0,30,pequeño


In [120]:

X= pd.get_dummies(X, columns=['range'])

X= pd.get_dummies(X, columns=['category'])


X= pd.get_dummies(X, columns=['main_category'])

X= pd.get_dummies(X, columns=['currency'])

X= pd.get_dummies(X, columns=['country'])



X.head()

,name,goal,backers,inUSDGoal,gap,range_grande,range_mediano,range_muy grande,range_muy pequeño,range_pequeño,category_3D Printing,category_Academic,category_Accessories,category_Action,category_Animals,category_Animation,category_Anthologies,category_Apparel,category_Apps,category_Architecture,category_Art,category_Art Books,category_Audio,category_Bacon,category_Blues,category_Calendars,category_Camera Equipment,category_Candles,category_Ceramics,category_Children's Books,category_Childrenswear,category_Chiptune,category_Civic Design,category_Classical Music,category_Comedy,category_Comic Books,category_Comics,category_Community Gardens,category_Conceptual Art,category_Cookbooks,...,main_category_Journalism,main_category_Music,main_category_Photography,main_category_Publishing,main_category_Technology,main_category_Theater,currency_AUD,currency_CAD,currency_CHF,currency_DKK,currency_EUR,currency_GBP,currency_HKD,currency_MXN,currency_NOK,currency_NZD,currency_SEK,currency_SGD,currency_USD,country_AT,country_AU,country_BE,country_CA,country_CH,country_DE,country_DK,country_ES,country_FR,country_GB,country_HK,country_IE,country_IT,country_LU,country_MX,country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
0,31,1000.0,0,1240.0,58,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,14,45000.0,3,45000.0,45,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,49,5000.0,1,5000.0,30,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,58,19500.0,14,19500.0,55,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,20,50000.0,224,50000.0,35,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [121]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,test_size=0.3, random_state=1)





In [122]:
len(Xtrain),len(Xtest),len(ytrain),len(ytest)
Xtrain.head()

,name,goal,backers,inUSDGoal,gap,range_grande,range_mediano,range_muy grande,range_muy pequeño,range_pequeño,category_3D Printing,category_Academic,category_Accessories,category_Action,category_Animals,category_Animation,category_Anthologies,category_Apparel,category_Apps,category_Architecture,category_Art,category_Art Books,category_Audio,category_Bacon,category_Blues,category_Calendars,category_Camera Equipment,category_Candles,category_Ceramics,category_Children's Books,category_Childrenswear,category_Chiptune,category_Civic Design,category_Classical Music,category_Comedy,category_Comic Books,category_Comics,category_Community Gardens,category_Conceptual Art,category_Cookbooks,...,main_category_Journalism,main_category_Music,main_category_Photography,main_category_Publishing,main_category_Technology,main_category_Theater,currency_AUD,currency_CAD,currency_CHF,currency_DKK,currency_EUR,currency_GBP,currency_HKD,currency_MXN,currency_NOK,currency_NZD,currency_SEK,currency_SGD,currency_USD,country_AT,country_AU,country_BE,country_CA,country_CH,country_DE,country_DK,country_ES,country_FR,country_GB,country_HK,country_IE,country_IT,country_LU,country_MX,country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
293345,22,25000.0,141,25000.0,45,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
235851,28,5000.0,234,5000.0,21,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
108081,13,2000.0,56,2480.0,30,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
197635,49,350.0,8,350.0,30,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
78503,59,2500.0,15,3100.0,45,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [123]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="modified_huber", penalty='l2', max_iter=5)
clf.fit(Xtrain, ytrain)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=5, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [124]:
result = np.array(clf.predict(Xtest))
testy = np.array(ytest)
aims=0
for i in range(0, len(result)-1):
  if result[i]==testy[i]:
    aims +=1

100*(aims/len(testy))





89.32420639867628

In [125]:
clf.intercept_
clf.coef_
#clf.predict_proba(Xtrain) #problem with hinge
clf.score(Xtest,ytest)


0.8932533971009894

In [126]:
dr1= pd.DataFrame(result)
dr1.head()

,0
0,0
1,1
2,0
3,0
4,0


In [127]:
from sklearn.metrics import classification_report , confusion_matrix

In [128]:
matrixConf = confusion_matrix(ytest, clf.predict(Xtest))

In [129]:
#A11 true negative prediction
#A21 false negative prediction
#A12 false positive prediction
#A22 true positive prediction

print(classification_report(ytest,clf.predict(Xtest)))

              precision    recall  f1-score   support

           0       0.95      0.88      0.91     56891
           1       0.81      0.92      0.86     31346

    accuracy                           0.89     88237
   macro avg       0.88      0.90      0.89     88237
weighted avg       0.90      0.89      0.89     88237



In [130]:
mcunf = np.array(matrixConf)
mcunf

array([[49960,  6931],
       [ 2488, 28858]])

In [131]:
mcunf = np.reshape(mcunf,(4))
mcunf
acc = (mcunf[3]+ mcunf[0])/len(ytest)
reCall = mcunf[3]/(mcunf[3]+ mcunf[1])
prec = mcunf[3]/(mcunf[3]+ mcunf[2])
prec, reCall, acc

(0.9206278313022396, 0.8063371426974769, 0.8932533971009894)

In [132]:
f1 = 2*prec*reCall/(prec+reCall)
f1

0.8597006032620839

In [133]:
from scipy import stats
a=clf.predict(Xtest)
neg = [ x for x in a if x != 1]
l= len(Xtest)
stats.binom_test(len(neg), n=l, p=0.70, alternative='greater')
# que la probabilidad de fallar sea mayor al 70 % es de :

0.9999999999999999

In [134]:
stats.describe(a)

DescribeResult(nobs=88237, minmax=(0, 1), mean=0.4056008250507157, variance=0.24109152808672113, skewness=0.3845118073581899, kurtosis=-1.8521506700021377)

In [135]:


result = np.array(clf.predict(Xtest))
dr = pd.DataFrame(Xtest)
dr['resultPredict']= dr1
dr['resultReal']= np.array(ytest)

dfinalAciertos = dr.query("resultPredict == resultReal")
dr.tail(10)

,name,goal,backers,inUSDGoal,gap,range_grande,range_mediano,range_muy grande,range_muy pequeño,range_pequeño,category_3D Printing,category_Academic,category_Accessories,category_Action,category_Animals,category_Animation,category_Anthologies,category_Apparel,category_Apps,category_Architecture,category_Art,category_Art Books,category_Audio,category_Bacon,category_Blues,category_Calendars,category_Camera Equipment,category_Candles,category_Ceramics,category_Children's Books,category_Childrenswear,category_Chiptune,category_Civic Design,category_Classical Music,category_Comedy,category_Comic Books,category_Comics,category_Community Gardens,category_Conceptual Art,category_Cookbooks,...,main_category_Photography,main_category_Publishing,main_category_Technology,main_category_Theater,currency_AUD,currency_CAD,currency_CHF,currency_DKK,currency_EUR,currency_GBP,currency_HKD,currency_MXN,currency_NOK,currency_NZD,currency_SEK,currency_SGD,currency_USD,country_AT,country_AU,country_BE,country_CA,country_CH,country_DE,country_DK,country_ES,country_FR,country_GB,country_HK,country_IE,country_IT,country_LU,country_MX,country_NL,country_NO,country_NZ,country_SE,country_SG,country_US,resultPredict,resultReal
62858,30,5000.0,0,5000.0,15,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,0
55574,49,2000.0,39,2000.0,32,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,1
146760,39,10000.0,2,10000.0,30,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,0
2654,52,1.0,9,1.0,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,1
201762,27,8000.0,60,8000.0,57,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,1
1178,46,1267.0,2,1267.0,6,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0
172290,30,100000.0,2,100000.0,30,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,0
149470,55,25000.0,3,28000.0,37,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN,0
281900,8,7000.0,0,5180.0,30,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0
153564,60,12000.0,2,12000.0,30,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,0


In [136]:
import re
dictResultSum={}
for column in dfinalAciertos.columns:

  x = re.findall("^cat", column)
  if x:
    dictResultSum[column]= dfinalAciertos[column].sum()
 


In [137]:
max_key = max(dictResultSum, key=dictResultSum.get)
min_key = min(dictResultSum, key=dictResultSum.get)
max_key ,min_key

('category_Product Design', 'category_Chiptune')

In [138]:
dictResultSum={}
for column in dfinalAciertos.columns:

  x = re.findall("^main", column)
  if x:
    dictResultSum[column]= dfinalAciertos[column].sum()
 

In [139]:
max_key = max(dictResultSum, key=dictResultSum.get)
min_key = min(dictResultSum, key=dictResultSum.get)
max_key ,min_key

('main_category_Film & Video', 'main_category_Dance')

In [140]:
dictResultSum={}
for column in dfinalAciertos.columns:

  x = re.findall("^range", column)
  if x:
    dictResultSum[column]= dfinalAciertos[column].sum()
 

In [141]:
max_key = max(dictResultSum, key=dictResultSum.get)
min_key = min(dictResultSum, key=dictResultSum.get)
max_key ,min_key

('range_muy grande', 'range_mediano')